In [46]:
import pyspark
from pyspark.sql import SparkSession
import requests

In [51]:
URL = 'https://drive.google.com/uc?export=download&id=1vcb_HBWsOSKW4XxhLfRpGlLzBLwHlGWJ'
output_file = 'multi_source_demo.zip'

# 发送 GET 请求并下载文件
response = requests.get(URL, allow_redirects=True)

# 检查请求是否成功
if response.status_code == 200:
    # 写入文件
    with open(output_file, 'wb') as f:
        f.write(response.content)
    print("文件下载成功。")
else:
    print("下载文件失败。")

文件下载成功。


In [43]:
!unzip multi_source_demo.zip

Archive:  multi_source_demo.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of multi_source_demo.zip or
        multi_source_demo.zip.zip, and cannot find multi_source_demo.zip.ZIP, period.
